In [192]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import Selection
import numpy as np
import os.path
from rdkit import Chem
import warnings
warnings.filterwarnings('ignore')

######## HELPER FUNCTIONS ###############
def extract_ligand_coordinate_sdf(ligand_input_file):
    """Extract ligand's atomic coordinates from input sdf file"""
    ligand = Chem.SDMolSupplier(ligand_input_file, removeHs = False)[0]
    conf = ligand.GetConformer()
    ligand_coord = conf.GetPositions()
    return np.array(ligand_coord)
    
def extract_ligand_coordinate_pdb(ligand_input_file):
    """Extract ligand's atomic coordinates from input pdb file"""
    parser = PDBParser()
    ligand_struct = parser.get_structure("lig", ligand_input_file)[0]
    ligand = Selection.unfold_entities(ligand_struct, 'R')[0]
    lig_coord = []
    for atom in ligand:
        lig_coord.append(atom.get_coord())
    return np.array(lig_coord)

def min_res_lig_distance(res, lig):
    """Returns the min distance between a residue and the ligand.
    Keyword arguments:
    res -- a numpy array of atomic coordinates of the residue
    lig -- a numpy array of atomic coordinates of the ligand
    Both numpy arrays should have shape of (n, 3), where n is the number of the atoms
    """
    lig_atm_num = len(lig)
    res_atm_num = len(res)
    # broadcasting the array of atomic coordinates of residue and ligand
    # so that the expanded arrays would have the same length
    res_expanded = np.repeat(res,repeats=lig_atm_num, axis=0)
    lig_expanded = np.tile(lig, (res_atm_num, 1))
    # computes the pairwise atomic distances between the residue and ligand
    # this would be much faster than using two for loops due to vectorization
    atom_dist = np.linalg.norm(res_expanded - lig_expanded, axis=1)
    # return the minimum atomic pairwise ditance between the residue and ligand
    # note that the numpy.min function here is faster than the min function due to vectorization
    return np.min(atom_dist)

def get_file_extension(file_path):
    """Extracts the file extension (in lowercase)"""
    _, extension = os.path.splitext(file_path)
    return extension.lower()

In [208]:
######## MAIN FUNCTION ###############
def check_residue_ligand_distances(input_protein, input_ligand, residue_list, distance_list):
    """Returns a list of booleans that indicate whether the ligand pose is within
    the ith distance in angstroms of the ith residue of the protein structure file.
    Keyword arguments:
    input_protein -- an input Protein structure file name
    input_ligand -- an input Ligand pose file name (only accept .pdb and .sdf files)
    residue_list -- a list of key residues' sequence index ids
    distance_list -- a list of cutoff distances (in angstroms)
    """
    # Check to make sure that the input files exist
    assert os.path.exists(input_protein), "the input protein structure file doesn't exist"
    assert os.path.exists(input_ligand), "the input ligand pose file doesn't exist"
    # Check to see if the sizes of the input residue list and the distance list are the same
    array_length = len(residue_list)
    assert array_length == len(distance_list), "the number of input residues is different from the number of input distances"

    # Extract atomic coordinates of ligand
    ligand_coord = []
    ligand_file_extension = get_file_extension(input_ligand)
    if ligand_file_extension == ".pdb":
        ligand_coord = extract_ligand_coordinate_pdb(input_ligand)
    elif ligand_file_extension == ".sdf":
        ligand_coord = extract_ligand_coordinate_sdf(input_ligand)
    else :
        raise Exception(" The input ligand file should be either a PDB or a SDF file")
    
    # parsing input protein pdb files
    parser = PDBParser()
    protein = parser.get_structure("protein", input_protein)[0]

    # Check if each residue is within the corresponding distance from the ligand
    protein_ress_num = len(Selection.unfold_entities(protein, 'R'))
    assert max(residue_list) <= protein_ress_num, "the input protein does not have residue %d" % max(residue_list)
    result_arr = []
    for id in range(array_length):
        resid = residue_list[id] - 1
        res = Selection.unfold_entities(protein, 'R')[resid]
        # Extract the atomic coordinates of each residue
        res_coord = []
        for atom in res:
            res_coord.append(atom.get_coord())
        # calculate min atomic pair-wise distance between the residue and the ligand 
        min_res_lig_dist = min_res_lig_distance(np.array(res_coord), ligand_coord)
        if min_res_lig_dist <= distance_list[id]:
            result_arr.append('True')
        else:
            result_arr.append('False')
    return result_arr
    

In [209]:
# Test case 1: an input receptor and an input ligand (pdb file)
# the min distance should be 4.7A (between atom O of res 36 and atom H18 of ligand)
check_residue_ligand_distances('test_cases/y4r_PP.pdb', 'test_cases/y4r_pam.pdb', [36, 36], [4, 5])   

['False', 'True']

In [210]:
# Test case 2: an input receptor and an input ligand (pdf file)
# the min distance of residue 36 should be 4.7A (between atom O of res 36P and atom H18 of ligand)
# the min distance of residue 101 should be 1.9A (between atom HG1 of res 65A and atom O2 of ligand)
check_residue_ligand_distances('test_cases/y4r_PP.pdb', 'test_cases/y4r_pam.sdf', [36, 36, 101, 101], [4, 5, 1, 2])

['False', 'True', 'False', 'True']

In [211]:
# Test case 3: an input receptor and an input ligand (pdf file), input residue id is outof bound
check_residue_ligand_distances('test_cases/y4r_PP.pdb', 'test_cases/y4r_pam.sdf', [36, 1000], [4, 5])

AssertionError: the input protein does not have residue 1000

In [212]:
# Test case 4: the size of input residue array is different from the size of the distance array
check_residue_ligand_distances('test_cases/y4r_PP.pdb', 'test_cases/y4r_pam.sdf', [36, 12, 11], [4, 5])

AssertionError: the number of input residues is different from the number of input distances